In [1]:
import torch
import torch.nn as nn
import math
torch.manual_seed(69)

In [83]:
vocab_size = 16
chars = "0123456789+_-=., " # '_':subtract      '-':minus      '.':EOS      ',':start token      ' ':padding
stoi = {c:i for i, c in enumerate(chars)}
encode = lambda str: [stoi[c] for c in str]
decode = lambda idxs: ''.join([chars[i] for i in idxs])

In [118]:
batch_size = 8
lhs_max = 999
# -999+-999=|.1998-, worst case
max_left = 10
max_right = 7

def get_batch():
    batch = [] 
    qs = [] # questions for encoder
    padding_mask_qs = []
    padding_mask_batch = []
    for _ in range(batch_size):
        a = torch.randint(-lhs_max, lhs_max+1, (2,))
        if torch.randint(0,2, (1,)).item():
            op = "_"
            result = (a[0]-a[1]).item()
        else:
            op = "+"
            result = a.sum().item()
        sign = "-" if result < 0 else "" # add negative sign to end   (SHOULD POSITIVE SIGN BE ADDED TOO?)

        q = f"{a[0]}{op}{a[1]}="
        ans = f".{abs(result)}{sign},"

        # padding
        padding_mask_qs.append([1]*len(q) + [0]*(max_left-len(q)))
        padding_mask_batch.append([1]*(len(ans)-1) + [0]*(max_right-len(ans)))
        q += " " * (max_left-len(q))
        ans = " " * (max_right-len(ans)) + ans
        print(q)
        print(ans)
        qs.append(encode(q))
        batch.append(ans)
        
    return torch.tensor(qs), get_examples(batch), torch.tensor(padding_mask_qs), torch.tensor(padding_mask_batch)

def get_examples(batch):
    Xb, Yb = [], []
    for x in batch:
        xb = x[:0:-1]
        yb = x[len(x)-2::-1]
        Xb.append(encode(xb))
        Yb.append(encode(yb))
    return torch.tensor(Xb), torch.tensor(Yb)


In [119]:
a, (b,c), d, e = get_batch()
d = d.view(d.shape[0], d.shape[1], 1)
print(a.shape)
print(b.shape)
print(c.shape)
print(d)
print(e.shape)


512+188=  
  .700,
686+408=  
 .1094,
-898_804= 
.1702-,
-500+766= 
  .266,
-771+719= 
  .52-,
31_-553=  
  .584,
744+157=  
  .901,
-902+52=  
 .850-,
torch.Size([8, 10])
torch.Size([8, 6])
torch.Size([8, 6])
tensor([[[1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [0],
         [0]],

        [[1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [0],
         [0]],

        [[1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [0]],

        [[1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [0]],

        [[1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [0]],

        [[1],
         [1],
         [1],
         [1],
         [1],
         [

In [120]:
print(encode(["."]))

[14]
